## Task 1
Import the CRM events and CRM call center logs tables into a PostgreSQL database. Use SQL to join the tables and summarize the average time to resolve complaints across a number of different dimensions.

Provide a short presentation / slidedeck (submitted as a PDF) that summarizes your findings.


In [1]:
import numpy as np
import pandas as pd

import psycopg2

In [2]:
crm_events = pd.read_csv("data/CRM Events.csv")
crm_logs = pd.read_csv("data/CRM Call Center Logs.csv")

In [3]:
crm_events.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Client_ID
0,2014-07-03,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,NaN,Email,2014-07-09,Closed with explanation,Yes,No,CR0922485,C00001925
1,2012-04-12,Bank account or service,Savings account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Email,2012-04-13,Closed with relief,Yes,No,CR0057298,C00003141
2,2012-04-03,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Email,2012-04-03,Closed without relief,Yes,No,CR0043811,C00000297
3,2012-03-14,Credit card,NaN,Billing disputes,NaN,NaN,NaN,NaN,Email,2012-03-14,Closed with relief,Yes,No,CR0035411,C00004275
4,2012-03-05,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Email,2012-03-06,Closed with relief,Yes,No,CR0030939,C00000900


In [4]:
crm_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23419 entries, 0 to 23418
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 23419 non-null  object 
 1   Product                       23419 non-null  object 
 2   Sub-product                   14091 non-null  object 
 3   Issue                         23419 non-null  object 
 4   Sub-issue                     0 non-null      float64
 5   Consumer complaint narrative  4467 non-null   object 
 6   Tags                          3276 non-null   object 
 7   Consumer consent provided?    6872 non-null   object 
 8   Submitted via                 23419 non-null  object 
 9   Date sent to company          23419 non-null  object 
 10  Company response to consumer  23419 non-null  object 
 11  Timely response?              23419 non-null  object 
 12  Consumer disputed?            22417 non-null  object 
 13  C

In [5]:
crm_logs

,Date received,Complaint ID,rand client,phonefinal,vru+line,call_id,priority,type,outcome,server,ser_start,ser_exit,ser_time
0,2017-03-27,CR2406263,C00004587,977-806-9726,AA0103,34536.0,0.0,NW,AGENT,MICHAL,13:34:11,13:40:23,00:06:12
1,2017-03-27,CR2405641,C00003328,322-598-7152,AA0205,34537.0,0.0,PS,AGENT,TOVA,10:58:22,11:16:10,00:17:48
2,2017-03-27,CR2405629,C00001685,508-311-5237,AA0110,34538.0,2.0,PS,AGENT,YIFAT,13:00:54,13:13:31,00:12:37
3,2017-03-23,CR2400594,C00001945,265-394-2727,AA0113,34540.0,2.0,PS,AGENT,AVNI,16:18:21,16:19:40,00:01:19
4,2017-03-22,CR2399607,C00004303,206-008-0460,AA0102,34541.0,1.0,PS,AGENT,STEREN,14:48:22,14:55:19,00:06:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2019-10-12,NaN,NaN,703-826-5722,NaN,NaN,NaN,NaN,NaN,NaN,15:48:32,16:07:39,00:19:07
3995,2014-11-03,NaN,NaN,387-514-1611,NaN,NaN,NaN,NaN,NaN,NaN,11:08:45,11:14:31,00:05:46
3996,2016-09-02,NaN,NaN,508-397-7936,NaN,NaN,NaN,NaN,NaN,NaN,13:08:09,13:24:05,00:15:56
3997,2015-02-03,NaN,NaN,883-529-9005,NaN,NaN,NaN,NaN,NaN,NaN,15:47:13,15:50:33,00:03:20


In [6]:
crm_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date received  3999 non-null   object 
 1   Complaint ID   2504 non-null   object 
 2   rand client    2504 non-null   object 
 3   phonefinal     3999 non-null   object 
 4   vru+line       3015 non-null   object 
 5   call_id        3015 non-null   float64
 6   priority       3015 non-null   float64
 7   type           3015 non-null   object 
 8   outcome        3015 non-null   object 
 9   server         3015 non-null   object 
 10  ser_start      3999 non-null   object 
 11  ser_exit       3999 non-null   object 
 12  ser_time       3999 non-null   object 
dtypes: float64(2), object(11)
memory usage: 406.3+ KB


## Data Processing

### Events data

In [7]:
crm_events

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Client_ID
0,2014-07-03,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,NaN,Email,2014-07-09,Closed with explanation,Yes,No,CR0922485,C00001925
1,2012-04-12,Bank account or service,Savings account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Email,2012-04-13,Closed with relief,Yes,No,CR0057298,C00003141
2,2012-04-03,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Email,2012-04-03,Closed without relief,Yes,No,CR0043811,C00000297
3,2012-03-14,Credit card,NaN,Billing disputes,NaN,NaN,NaN,NaN,Email,2012-03-14,Closed with relief,Yes,No,CR0035411,C00004275
4,2012-03-05,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Email,2012-03-06,Closed with relief,Yes,No,CR0030939,C00000900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23414,2017-03-23,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Phone,2017-03-23,Closed with explanation,Yes,NaN,CR2400594,C00001945
23415,2017-03-24,Bank account or service,Savings account,Deposits and withdrawals,NaN,NaN,Older American,NaN,Phone,2017-03-24,Closed with explanation,Yes,NaN,CR2403628,C00002421
23416,2017-03-27,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,Older American,NaN,Phone,2017-03-27,In progress,Yes,NaN,CR2405629,C00001685
23417,2017-03-27,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,NaN,Phone,2017-03-27,In progress,Yes,NaN,CR2405641,C00003328


In [8]:
crm_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23419 entries, 0 to 23418
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 23419 non-null  object 
 1   Product                       23419 non-null  object 
 2   Sub-product                   14091 non-null  object 
 3   Issue                         23419 non-null  object 
 4   Sub-issue                     0 non-null      float64
 5   Consumer complaint narrative  4467 non-null   object 
 6   Tags                          3276 non-null   object 
 7   Consumer consent provided?    6872 non-null   object 
 8   Submitted via                 23419 non-null  object 
 9   Date sent to company          23419 non-null  object 
 10  Company response to consumer  23419 non-null  object 
 11  Timely response?              23419 non-null  object 
 12  Consumer disputed?            22417 non-null  object 
 13  C

In [9]:
crm_events["Date received"] = pd.to_datetime(crm_events["Date received"])
crm_events["Date sent to company"] = pd.to_datetime(crm_events["Date sent to company"])

In [10]:
cols = "Consumer complaint narrative"
crm_events[cols].apply(lambda x: len(x) if isinstance(x,str) else x).sort_values(ascending=False)

crm_events.drop(cols,axis = 1,inplace=True)

In [11]:
crm_events["Sub-issue"].value_counts()

Series([], Name: count, dtype: int64)

this variable, Sub-issue, will be dropped later

In [12]:
crm_events.drop("Sub-issue", axis=1, inplace=True)

## logs

In [13]:
crm_logs["Date received"] = pd.to_datetime(crm_logs["Date received"])

In [14]:
crm_logs[["call_id", "priority"]] = crm_logs[["call_id", "priority"]].astype(str)
crm_logs["call_id"] = crm_logs["call_id"].apply(lambda x: int(float(x)) if isinstance(float(x),float) and pd.notnull(float(x)) else x)
crm_logs["priority"] = crm_logs["priority"].apply(lambda x: int(float(x)) if isinstance(float(x),float) and pd.notnull(float(x)) else x)

In [15]:
crm_logs

,Date received,Complaint ID,rand client,phonefinal,vru+line,call_id,priority,type,outcome,server,ser_start,ser_exit,ser_time
0,2017-03-27,CR2406263,C00004587,977-806-9726,AA0103,34536,0,NW,AGENT,MICHAL,13:34:11,13:40:23,00:06:12
1,2017-03-27,CR2405641,C00003328,322-598-7152,AA0205,34537,0,PS,AGENT,TOVA,10:58:22,11:16:10,00:17:48
2,2017-03-27,CR2405629,C00001685,508-311-5237,AA0110,34538,2,PS,AGENT,YIFAT,13:00:54,13:13:31,00:12:37
3,2017-03-23,CR2400594,C00001945,265-394-2727,AA0113,34540,2,PS,AGENT,AVNI,16:18:21,16:19:40,00:01:19
4,2017-03-22,CR2399607,C00004303,206-008-0460,AA0102,34541,1,PS,AGENT,STEREN,14:48:22,14:55:19,00:06:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2019-10-12,NaN,NaN,703-826-5722,NaN,nan,nan,NaN,NaN,NaN,15:48:32,16:07:39,00:19:07
3995,2014-11-03,NaN,NaN,387-514-1611,NaN,nan,nan,NaN,NaN,NaN,11:08:45,11:14:31,00:05:46
3996,2016-09-02,NaN,NaN,508-397-7936,NaN,nan,nan,NaN,NaN,NaN,13:08:09,13:24:05,00:15:56
3997,2015-02-03,NaN,NaN,883-529-9005,NaN,nan,nan,NaN,NaN,NaN,15:47:13,15:50:33,00:03:20


## Init Databse

In [25]:
crm_events.columns = [i.lower().replace(' ', '_').replace('-', '_').replace("?", "") for i in crm_events.columns]
crm_logs.columns = [i.lower().replace(' ', '_').replace('-', '_').replace("+", "_") for i in crm_logs.columns]

In [26]:
print(crm_events.columns, crm_logs.columns)

Index(['date_received', 'product', 'sub_product', 'issue', 'tags',
       'consumer_consent_provided', 'submitted_via', 'date_sent_to_company',
       'company_response_to_consumer', 'timely_response', 'consumer_disputed',
       'complaint_id', 'client_id'],
      dtype='object') Index(['date_received', 'complaint_id', 'rand_client', 'phonefinal',
       'vru_line', 'call_id', 'priority', 'type', 'outcome', 'server',
       'ser_start', 'ser_exit', 'ser_time'],
      dtype='object')


In [27]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="mobile_bank",
    user="postgres",
    password="bandung1",
    host="localhost",
    port="5433"
)
# Create a cursor object
cur = conn.cursor()


In [28]:
# Get column names and data types from the DataFrame
column_names_crm_events = crm_events.columns.tolist()
data_types_crm_events = crm_events.dtypes.tolist()



column_names_crm_logs = crm_logs.columns.tolist()
data_types_crm_logs = crm_logs.dtypes.tolist()

for _, i in enumerate(zip(column_names_crm_events, data_types_crm_events)):
    print(_, i[0], i[1])
print("==="*20)
for _, i in enumerate(zip(column_names_crm_logs, data_types_crm_logs)):
    print(_, i[0], i[1], sep="\t")

0 date_received datetime64[ns]
1 product object
2 sub_product object
3 issue object
4 tags object
5 consumer_consent_provided object
6 submitted_via object
7 date_sent_to_company datetime64[ns]
8 company_response_to_consumer object
9 timely_response object
10 consumer_disputed object
11 complaint_id object
12 client_id object
0	date_received	datetime64[ns]
1	complaint_id	object
2	rand_client	object
3	phonefinal	object
4	vru_line	object
5	call_id	object
6	priority	object
7	type	object
8	outcome	object
9	server	object
10	ser_start	object
11	ser_exit	object
12	ser_time	object


In [29]:
###  CONFIG

data_type = {
    "CRM_events": ["VARCHAR(255)" if i not in ["date_received", "date_sent_to_company"] else "DATE" for i in crm_events.columns],
    "call_center_logs":
        ["VARCHAR(255)" if i not in ["date_received"] else "DATE" for i in crm_logs.columns[:-3]]
}
# adding for ser_start, ser_exit, ser_time
add = ["TIME", "TIME", "TIME"]
for d in add:
    data_type["call_center_logs"].append(d)
data = {
    "CRM_events":{
        "data":crm_events,
        "cols_name":column_names_crm_events,
        "cols_type":data_type["CRM_events"]
    },
    "call_center_logs":{
        "data":crm_logs,
        "cols_name":column_names_crm_logs,
        "cols_type":data_type["call_center_logs"]
    }
}


creating database

In [30]:
for d in data.keys():
    try:
        # Construct the CREATE TABLE statement
        create_table_query = f"CREATE TABLE IF NOT EXISTS {d} ("
        for column_name, data_type in zip(data[d]["cols_name"], data[d]["cols_type"]):
            create_table_query += f'\n"{column_name}" {data_type},'
    
        create_table_query = create_table_query[:-1] + ")"
        print(create_table_query)
        # Execute the CREATE TABLE statement
        cur.execute(create_table_query)
        conn.commit()
    # Your code to execute SQL queries goes here
    except psycopg2.Error as e:
        print("Error:", e)
        conn.rollback()  # Roll back the transaction to its starting point
    print("\n")
    

CREATE TABLE IF NOT EXISTS CRM_events (
"date_received" DATE,
"product" VARCHAR(255),
"sub_product" VARCHAR(255),
"issue" VARCHAR(255),
"tags" VARCHAR(255),
"consumer_consent_provided" VARCHAR(255),
"submitted_via" VARCHAR(255),
"date_sent_to_company" DATE,
"company_response_to_consumer" VARCHAR(255),
"timely_response" VARCHAR(255),
"consumer_disputed" VARCHAR(255),
"complaint_id" VARCHAR(255),
"client_id" VARCHAR(255))


CREATE TABLE IF NOT EXISTS call_center_logs (
"date_received" DATE,
"complaint_id" VARCHAR(255),
"rand_client" VARCHAR(255),
"phonefinal" VARCHAR(255),
"vru_line" VARCHAR(255),
"call_id" VARCHAR(255),
"priority" VARCHAR(255),
"type" VARCHAR(255),
"outcome" VARCHAR(255),
"server" VARCHAR(255),
"ser_start" TIME,
"ser_exit" TIME,
"ser_time" TIME)


inserting database values

In [31]:
crm_events.to_csv("data/terbaru_crm.csv", index=False)
crm_logs.to_csv("data/terbaru_logs.csv", index=False)

In [32]:
for d in data.keys():
    try:
        # Insert data into the table
        for row in data[d]["data"].itertuples(index=False, name=None):
            insert_query = f"INSERT INTO {d} VALUES ( {row} )"
            try:
                #cur.execute(insert_query, row)
                print(insert_query, row)
            except psycopg2.Error as e:
                print(f"Error inserting row into table {d}: {e}")
        # Commit the transaction
        conn.commit()
    except psycopg2.Error as e:
        print("Error:", e)
        conn.rollback()  # Roll back the transaction to its starting point

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

